In [576]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
mpl.rcParams['axes.unicode_minus'] = False
plt.rc('font', family='Malgun Gothic')
import re

In [577]:
df_seoul = pd.read_csv('data/서울공원.csv', encoding='EUC-KR')

In [593]:
df_seoul.columns


Index(['연번', '공원명', '공원개요', '면적', '개원일', '주요시설', '주요식물', '안내도', '오시는길',
       '이용시참고사항', '이미지', '지역', '공원주소', '관리부서', '전화번호', 'X좌표(GRS80TM)',
       'Y좌표(GRS80TM)', 'X좌표(WGS84)', 'Y좌표(WGS84)', '바로가기'],
      dtype='object')

In [579]:
df_seoul.면적[0].split('㎡')[0]

'2896887'

In [580]:
df = pd.DataFrame()

In [581]:
x = '[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》ㄱ-ㅎㅏ-ㅣ가-힣㎡㎥m² ]'

In [582]:
mm = []
for i in df_seoul.면적.values:
    try:
        place = re.sub(x,'',i.split()[0].strip())
        print(place)
        mm.append(float(place))
    except:#한글만 들어가있던가 0인경우
        mm.append(0)
        pass

2896887
80683
9132690
480994
2284085
8948.1

6456
80309
1980.4
5197.7
11467
229539
15179.7
16734.60
61544
424106
560552
26696.8
75900
113021.72
201779
258991
10420819.08
171294.1

297926
411972
98470
2038741.3
603715.2
108120
29619.3
29843.8
126026
135002.7
384026
180000
416031
160720
2277252
19599.70
138596
185295
254648
29816.5
762642
664905
99599.9
5802

585652
649709.6
72132
66027.8
265582
2088704.9
112070
993724
219167
1349556.4
1511198
1097565
328018

207790
1018689
954553
1667848
13224
130345
5332422
5107964
47832
213552
168099.87
359435.2
36660
380735
1807420.9
359342.8
632733
328018
6691885.3
285757
110407
29682.2
137759
917190
51470
662543
217946
179666
34955
18083
4835
19650
7000
10712
17182
160349.3
87432.4
13467
46871
49972
74385
8868
10634
22113
1082847
3692455
386500
23450
2400833
2752421
4180551
147859
111650
519782
59347
25500
2560
32000
45615
631021
109635
140022
211392
157535


In [583]:
df['공원명'] = df_seoul['공원명']
df['지역'] = df_seoul['지역']
df['면적'] = mm
df['경도'] = df_seoul['X좌표(WGS84)']
df['위도'] = df_seoul['Y좌표(WGS84)']

In [584]:
df.면적

0      2896887.0
1        80683.0
2      9132690.0
3       480994.0
4      2284085.0
         ...    
127     109635.0
128     140022.0
129     211392.0
130     157535.0
131          0.0
Name: 면적, Length: 132, dtype: float64

In [585]:
#df['면적분류'] = df['면적'].agg(lambda x : '소형' if x<100000 else '중형'  )

In [586]:
park_size, park_size_i = [], []
for i in df.면적:
    if i ==0:
        park_size.append(np.nan), park_size_i.append(np.nan)
    elif i <= 100000:
        park_size.append('소형'), park_size_i.append(3)
    elif i <= 1000000:
        park_size.append('중형'), park_size_i.append(7)
    else:
        park_size.append('대형'), park_size_i.append(15)

df['분류'] = park_size
df['크기'] = park_size_i
df.tail()


,공원명,지역,면적,경도,위도,분류,크기
127,용마도시자연공원(사가정공원),중랑구,109635.0,127.095997,37.579762,중형,7.0
128,문화비축기지,마포구,140022.0,126.893246,37.571718,중형,7.0
129,경춘선숲길,NaN,211392.0,127.076482,37.627077,중형,7.0
130,율현공원,강남구,157535.0,127.115594,37.472332,중형,7.0
131,서울로7017,중구,0.0,NaN,NaN,NaN,NaN


In [587]:
import folium
total_df = df.dropna()
df = total_df

In [588]:
map = folium.Map(location=[37.541, 126.986], zoom_start=9.5)

for i in df.index:
    folium.Circle(
        radius= int(df['크기'][i])*100, #int 64는 처리 불가
        location = [float(df.위도[i]), float(df.경도[i])],
        popup = folium.Popup(f'{ df["면적"][i]}', max_width=200),
        tooltip = df.공원명[i],
        color= 'crimson',
        fillcolor = 'crimson'
    ).add_to(map)
map

In [589]:
from folium.plugins import MarkerCluster

In [590]:
locations, names = [], []
for i in df.index:
    locations.append([df.위도[i], df.경도[i]])
    names.append(df.공원명[i])
    

In [591]:
map = folium.Map(location=[37.541, 126.986], zoom_start=9.5)
MarkerCluster(
    locations=locations, popups=names,
    name='서울공원', overlay=True, control=True).add_to(map)

map

In [592]:
df.to_csv('data/서울공원분석.csv')